### 이미지 데이터 csv 만들기

In [1]:
import csv
from pathlib import Path

import cv2
import numpy as np


In [2]:
# =========================
# 설정
# =========================
data_root = Path(r"C:\Users\kdt008\face_imgs_gender_balanced")      # 연령대 폴더 루트
out_csv   = Path(r"./Data/img_face.csv")

IMG_SIZE = (128, 128)  # (width, height)

CLASSES = [
    "infant",
    "child",
    "teen",
    "young_adult",
    "middle_aged",
    "senior",
    "elderly"
]

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# =========================
# (중요) 연령대 폴더 없으면 생성
# =========================
data_root.mkdir(parents=True, exist_ok=True)

for cls in CLASSES:
    cls_dir = data_root / cls
    cls_dir.mkdir(parents=True, exist_ok=True)

# =========================
# CSV 헤더
# =========================
n_features = IMG_SIZE[0] * IMG_SIZE[1]
header = [f"p{i}" for i in range(n_features)] + ["target"]

# =========================
# 메인
# =========================
total = 0
skipped = 0

with open(out_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for cls in CLASSES:
        cls_dir = data_root / cls

        img_files = [
            p for p in cls_dir.iterdir()
            if p.is_file() and p.suffix.lower() in IMG_EXTS
        ]

        if len(img_files) == 0:
            print(f"[INFO] 이미지 없음 → 스킵: {cls}")
            continue

        for img_path in img_files:
            # 1) 그레이스케일 로드
            gray = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
            if gray is None:
                skipped += 1
                continue

            # 2) 리사이즈
            gray = cv2.resize(gray, IMG_SIZE, interpolation=cv2.INTER_AREA)

            # 3) 정규화 (0~1)
            x = gray.astype(np.float32) / 255.0

            # 4) 1차원 벡터로 변환
            row = x.reshape(-1).tolist()
            row.append(cls)  # target = 연령대

            writer.writerow(row)
            total += 1

print("====== DONE ======")
print("CSV 저장 위치:", out_csv)
print("총 저장 이미지:", total)
print("스킵:", skipped)
print("특징 수:", n_features)

====== DONE ======
CSV 저장 위치: Data\img_face.csv
총 저장 이미지: 2559
스킵: 0
특징 수: 16384
